# Сбор данных

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import os

# Прочтение датасеты с результатами матчей
db = sqlite3.connect('football_stats.db')
conn = sqlite3.connect('football_stats.db')
df = pd.read_sql("SELECT DISTINCT * FROM football_stats ORDER BY start_datetime", conn)

conn.commit()
conn.close()

In [2]:
df

,region,tournament,stage,start_date,start_time,start_datetime,home,guest,goal_home,goal_guest,...,save_home,save_guest,foul_home,foul_guest,yellow_home,yellow_guest,xG_home,xG_guest,red_home,red_guest
0,ЕВРОПА,ЛИГА ЧЕМПИОНОВ,1/2,25.06.2019,16:00,2019-06-25 16:00:00,Тре Пенне (Сан),Санта-Колома (Анд),0.0,1.0,...,4.0,2.0,8.0,3.0,0.0,0.0,NaN,NaN,0.0,0.0
1,ЕВРОПА,ЛИГА ЧЕМПИОНОВ,1/2,25.06.2019,21:45,2019-06-25 21:45:00,Фероникели (Кос),Линкольн Ред Импс (Гиб),1.0,0.0,...,1.0,2.0,7.0,10.0,4.0,4.0,NaN,NaN,0.0,0.0
2,ЕВРОПА,ЛИГА ЕВРОПЫ,ФИНАЛ,27.06.2019,20:30,2019-06-27 20:30:00,Барри (Уэл),Клифтонвилл (Сир),0.0,0.0,...,2.0,0.0,10.0,6.0,1.0,1.0,NaN,NaN,0.0,0.0
3,ЕВРОПА,ЛИГА ЕВРОПЫ,ФИНАЛ,27.06.2019,20:30,2019-06-27 20:30:00,Прогрес Нидеркорн (Люк),Кардифф Метрополитан (Уэл),1.0,0.0,...,0.0,5.0,12.0,5.0,1.0,1.0,NaN,NaN,0.0,0.0
4,ЕВРОПА,ЛИГА ЕВРОПЫ,ФИНАЛ,27.06.2019,21:00,2019-06-27 21:00:00,Клаксвик (Фар),Тре Фиори (Сан),5.0,1.0,...,3.0,5.0,15.0,11.0,4.0,3.0,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14343,ИСПАНИЯ,ПРИМЕРА,ТУР 11,27.10.2024,20:30,2024-10-27 20:30:00,Бетис,Атлетико,1.0,0.0,...,3.0,2.0,8.0,4.0,3.0,2.0,1.95,0.54,0.0,0.0
14344,ГЕРМАНИЯ,БУНДЕСЛИГА,ТУР 8,27.10.2024,21:30,2024-10-27 21:30:00,Хайденхайм,Хоффенхайм,0.0,0.0,...,4.0,3.0,12.0,16.0,1.0,2.0,1.29,0.56,0.0,0.0
14345,ИТАЛИЯ,СЕРИЯ А,ТУР 9,27.10.2024,22:45,2024-10-27 22:45:00,Фиорентина,Рома,5.0,1.0,...,4.0,5.0,13.0,11.0,2.0,5.0,3.80,0.65,0.0,1.0
14346,ФРАНЦИЯ,ПЕРВАЯ ЛИГА,ТУР 9,27.10.2024,22:45,2024-10-27 22:45:00,Марсель,ПСЖ,0.0,3.0,...,5.0,1.0,4.0,7.0,1.0,0.0,0.22,2.98,1.0,0.0


In [3]:
df.columns

Index(['region', 'tournament', 'stage', 'start_date', 'start_time',
       'start_datetime', 'home', 'guest', 'goal_home', 'goal_guest',
       'possession_home', 'possession_guest', 'shot_home', 'shot_guest',
       'shot_on_target_home', 'shot_on_target_guest', 'shot_miss_home',
       'shot_miss_guest', 'free_kick_home', 'free_kick_guest', 'corner_home',
       'corner_guest', 'offside_home', 'offside_guest', 'save_home',
       'save_guest', 'foul_home', 'foul_guest', 'yellow_home', 'yellow_guest',
       'xG_home', 'xG_guest', 'red_home', 'red_guest'],
      dtype='object')

# Метод анализа игры команд в атаке и в обороне

* Гипотеза следующего исследования состоит в том, что по статистике по офсайдам можно понять, как высоко играет в обороне команда, и как часто она создает офсайдные ловушки для соперников. Поэтому когда я буду считать количество офсайдов команды А в обороне, то есть для расчётов нужно брать количестов офсайдов соперника команды А.
* Тем не менее, игра в атаке команды так же важна. Поэтому так же должен быть функционал, который бы считал значения офсайдов у команда А в атаке, и поэтому для этого нужно создать функцию, которая бы считала значения офсайдов как в атаке, так и в обороне. 

**Реализацию подобной функции можно увидеть ниже**

In [14]:
def make_offsides_stats(df, teams, matches_start, matches_end=None, defend=True):
   
    # Здесь сохранятся данные (средняя, медиана, минимум, максимум, 20% квантиль, 80% квантиль
    means = []
    medians = []
    mins = []
    maxes = []
    quantiles_20 = []
    quantiles_80 = []
    
    # Проход по всем командам, которые есть в переменной teams
    for team in teams:
        
        # Сбор значения из защиты
        def get_def_offsides(row):
            if row['home'] == team:
                return row['offside_guest']
            elif row['guest'] == team:
                return row['offside_home']

        # Сбор значения из аттаки
        def get_att_offsides(row):
            if row['home'] == team:
                return row['offside_home']  
            elif row['guest'] == team:
                return row['offside_guest']

        # отдельный датасет с играми по каждой анализируемой команде
        df_team = df[(df['home'] == team) | (df['guest'] == team)]
    
        # Если у команды меньше или ровно 3 игр, то не считаем статистику, так как мало данных
        if df_team.shape[0] <= 3:
            mins.append(None)
            medians.append(None)
            means.append(None)
            quantiles_20.append(None)
            quantiles_80.append(None)
            maxes.append(None)
            continue

        # Считаем разные офсайды по команде, в зависимости от того, что нас интересует - игра в обороне или в атаке за последние N матчей (обычно N=5)
        if defend:
            offsides = df_team.apply(get_def_offsides, axis=1)[matches_start:matches_end]
        else:
            offsides = df_team.apply(get_att_offsides, axis=1)[matches_start:matches_end]


        # Добавляем нужные статистики по каждой команде за последние N матчей
        means.append(round(offsides.mean(), 2))
        medians.append(round(offsides.median(), 2))
        quantiles_20.append(round(offsides.quantile(q=0.2), 2))
        quantiles_80.append(round(offsides.quantile(q=0.8), 2))
        mins.append(round(offsides.min(), 2))
        maxes.append(round(offsides.max(), 2))

    # Датасет по всем командам и всем статистикам
    res = pd.DataFrame(
                {'Команда': teams,
                 'Минимум': mins,
                 '20% квантиль': quantiles_20,
                 'Медиана': medians,
                 'Среднее': means,
                 '80% квантиль': quantiles_80,
                 'Максимум': maxes}
              ).sort_values(by=['Среднее'], ascending=False).reset_index(drop=True).dropna()

    # Делаем синтетические матч по всем командам, в которых статистики по оффсайдам по синтетическим матч - это сумма значений по каждой команде в матче 
    df_merged = res.merge(res, how='cross')
    df_merged = df_merged[df_merged['Команда_x'] != df_merged['Команда_y']]
    df_merged['Минимум'] = df_merged['Минимум_x'] + df_merged['Минимум_y']
    df_merged['20% квантиль'] = df_merged['20% квантиль_x'] + df_merged['20% квантиль_y']
    df_merged['Медиана'] = df_merged['Медиана_x'] + df_merged['Медиана_y']
    df_merged['Среднее'] = df_merged['Среднее_x'] + df_merged['Среднее_y']
    df_merged['80% квантиль'] = df_merged['80% квантиль_x'] + df_merged['80% квантиль_y']
    df_merged['Максимум'] = df_merged['Максимум_x'] + df_merged['Максимум_y']
    df_merged = df_merged[['Команда_x', 'Команда_y', 'Минимум', '20% квантиль', 'Медиана', 'Среднее', '80% квантиль', 'Максимум']]

    df_merged = df_merged.rename(columns={'Команда_x': 'home', 'Команда_y': 'guest'})
    
    return df_merged

In [15]:
# Оставляем только матчи Топ-5 лиг с сезона 2019/2020
df_5 = df[~df['region'].isin(['ЕВРОПА', 'РОССИЯ'])]

# Оставляем только команды, которые играют в Топ-5 лиг в сезоне 2024/2025
teams_5 = df_5[df_5['start_datetime'] > '2024-06-15']['home'].unique()

In [16]:
# Анализ последних 5 матчей по всем команам из Топ-5 лиг, согласно игре команд в обороне
df_def_stats = make_offsides_stats(df_5, teams_5, -5, defend=True)
df_def_stats

,home,guest,Минимум,20% квантиль,Медиана,Среднее,80% квантиль,Максимум
1,Барселона,Марсель,4.0,8.8,14.0,13.0,17.6,20.0
2,Барселона,Парма,5.0,9.0,14.0,12.6,15.8,19.0
3,Барселона,Валенсия,3.0,7.8,13.0,12.4,17.0,21.0
4,Барселона,Осасуна,3.0,9.4,15.0,12.4,16.2,17.0
5,Барселона,Лас-Пальмас,5.0,9.0,14.0,12.4,16.2,17.0
...,...,...,...,...,...,...,...,...
8458,Анже,Хоффенхайм,0.0,0.0,0.0,0.6,1.2,2.0
8459,Анже,Наполи,0.0,0.0,0.0,0.4,0.4,2.0
8460,Анже,Жирона,0.0,0.0,0.0,0.4,0.4,2.0
8461,Анже,Эспаньол,0.0,0.0,0.0,0.4,0.4,2.0


In [18]:
# Анализ последних 5 матчей по всем команам из Топ-5 лиг, согласно игре команд в атаке
df_att_stats = make_offsides_stats(df_5, teams_5, -5, defend=False)
df_att_stats

,home,guest,Минимум,20% квантиль,Медиана,Среднее,80% квантиль,Максимум
1,Хетафе,Алавес,2.0,4.4,7.0,8.0,10.0,18.0
2,Хетафе,ПСЖ,2.0,3.6,6.0,7.4,11.0,15.0
3,Хетафе,Ноттингем Форест,2.0,4.4,7.0,7.4,10.0,14.0
4,Хетафе,Осасуна,1.0,2.6,6.0,7.2,10.6,17.0
5,Хетафе,Реал Мадрид,1.0,3.4,5.0,7.2,9.4,19.0
...,...,...,...,...,...,...,...,...
8458,Манчестер Сити,Лас-Пальмас,0.0,0.0,1.0,1.2,2.2,3.0
8459,Манчестер Сити,Хоффенхайм,0.0,0.0,0.0,1.2,2.4,4.0
8460,Манчестер Сити,Монца,0.0,0.0,1.0,1.0,2.0,2.0
8461,Манчестер Сити,Ювентус,0.0,0.0,0.0,1.0,2.2,3.0


**Вывод:** Таким образом, мы получили статистику по офсайдам, полученных командой в обороне и в атаке за последние N матчей. Теперь перейдем к определению способа, как из этих двух таблиц получить 1 прогноз на матч

# Получение прогноза по синтетическим матчам

Методом перебора я остановился на 3 вариантах агрегации этих 2 таблиц по атаке и по обороне:
1) Сумма
2) Средняя
3) Минимум

**Начнем с суммы**

In [20]:
df_final_sum = pd.merge(df_att_stats,df_def_stats,on=['home', 'guest'], how='inner', suffixes= ('_att', '_def'))
                             
df_final_sum['20% квантиль'] = df_final_sum['20% квантиль_att'] + df_final_sum['20% квантиль_def']
df_final_sum['Медиана'] = df_final_sum['Медиана_att'] + df_final_sum['Медиана_def']
df_final_sum['Среднее'] = df_final_sum['Среднее_att'] + df_final_sum['Среднее_def']
df_final_sum['80% квантиль'] = df_final_sum['80% квантиль_att'] + df_final_sum['80% квантиль_def']
df_final_sum = df_final_sum[['home', 'guest', '20% квантиль', 'Медиана', 'Среднее', '80% квантиль']]

df_final_sum

,home,guest,20% квантиль,Медиана,Среднее,80% квантиль
0,Хетафе,Алавес,6.2,11.0,12.4,16.6
1,Хетафе,ПСЖ,5.4,9.0,11.0,16.4
2,Хетафе,Ноттингем Форест,5.2,10.0,10.4,14.6
3,Хетафе,Осасуна,6.6,13.0,13.2,19.0
4,Хетафе,Реал Мадрид,5.2,9.0,11.2,15.0
...,...,...,...,...,...,...
8367,Манчестер Сити,Лас-Пальмас,3.8,6.0,6.6,9.4
8368,Манчестер Сити,Хоффенхайм,1.0,1.0,3.2,5.6
8369,Манчестер Сити,Монца,1.8,3.0,3.4,5.2
8370,Манчестер Сити,Ювентус,1.0,1.0,3.2,5.6


**Попробуем усреднить значения по атаке и офсайдам**

In [19]:
df_final_mean = pd.merge(df_att_stats, df_def_stats,on=['home', 'guest'], how='inner', suffixes= ('_att', '_def'))

df_final_mean['20% квантиль'] = np.mean((df_final_mean['20% квантиль_att'], df_final_mean['20% квантиль_def']), axis=0)
df_final_mean['Медиана'] = np.mean((df_final_mean['Медиана_att'], df_final_mean['Медиана_def']), axis=0)
df_final_mean['Среднее'] = np.mean((df_final_mean['Среднее_att'], df_final_mean['Среднее_def']), axis=0)
df_final_mean['80% квантиль'] = np.mean((df_final_mean['80% квантиль_att'], df_final_mean['80% квантиль_def']), axis=0)
df_final_mean = df_final_mean[['home', 'guest', '20% квантиль', 'Медиана', 'Среднее', '80% квантиль']]

df_final_mean

,home,guest,20% квантиль,Медиана,Среднее,80% квантиль
0,Ноттингем Форест,Хетафе,2.7,4.5,5.1,7.2
1,Ноттингем Форест,Леганес,2.8,4.0,5.0,6.8
2,Ноттингем Форест,Вильярреал,2.4,4.0,4.6,6.5
3,Ноттингем Форест,Боруссия М,3.3,3.5,4.9,6.3
4,Ноттингем Форест,Монако,3.7,4.0,5.1,6.9
...,...,...,...,...,...,...
8367,Манчестер Сити,Хоффенхайм,0.5,2.5,2.2,3.0
8368,Манчестер Сити,Милан,1.9,3.5,3.4,4.1
8369,Манчестер Сити,Брест,1.3,2.5,3.0,4.2
8370,Манчестер Сити,Монца,0.9,2.5,2.2,2.9


**И заканчиваем минимумом по обороне и атаке**

In [20]:
df_final_min = pd.merge(df_att_stats, df_def_stats,on=['home', 'guest'], how='inner', suffixes= ('_att', '_def'))

df_final_min['20% квантиль'] = np.min((df_final_min['20% квантиль_att'], df_final_min['20% квантиль_def']), axis=0)
df_final_min['Медиана'] = np.min((df_final_min['Медиана_att'], df_final_min['Медиана_def']), axis=0)
df_final_min['Среднее'] = np.min((df_final_min['Среднее_att'], df_final_min['Среднее_def']), axis=0)
df_final_min['80% квантиль'] = np.min((df_final_min['80% квантиль_att'], df_final_min['80% квантиль_def']), axis=0)
df_final_min = df_final_min[['home', 'guest', '20% квантиль', 'Медиана', 'Среднее', '80% квантиль']]

df_final_min

,home,guest,20% квантиль,Медиана,Среднее,80% квантиль
0,Ноттингем Форест,Хетафе,0.0,1.0,2.0,3.6
1,Ноттингем Форест,Леганес,0.8,2.0,2.0,3.2
2,Ноттингем Форест,Вильярреал,0.0,1.0,1.2,2.2
3,Ноттингем Форест,Боруссия М,0.8,1.0,1.8,3.2
4,Ноттингем Форест,Монако,1.8,2.0,2.4,3.2
...,...,...,...,...,...,...
8367,Манчестер Сити,Хоффенхайм,0.0,1.0,1.2,1.6
8368,Манчестер Сити,Милан,0.0,0.0,1.0,1.0
8369,Манчестер Сити,Брест,0.0,0.0,1.0,1.6
8370,Манчестер Сити,Монца,0.0,1.0,1.0,1.4


**Вывод:** Методы получения прогноза данных есть, теперь выберем лучшие способы с помощью максимизации вероятности на тестовых данных

# Проверка точности модели

Проверять точность модели будет на матчах, которые проходили с 18 по 21 октября. Соответственно нужно обучать модель на матчах, которые проходили до 18 октября.

In [22]:
df_def_stats = make_offsides_stats(df_5, teams_5, -7, -2, defend=True)
df_def_stats

,home,guest,Минимум,20% квантиль,Медиана,Среднее,80% квантиль,Максимум
1,Барселона,Брайтон,10.0,10.0,11.0,12.8,16.2,17.0
2,Барселона,Боруссия Д,8.0,9.6,10.0,12.2,15.0,19.0
3,Барселона,Валенсия,8.0,8.8,9.0,12.2,16.0,20.0
4,Барселона,Фулхэм,7.0,8.6,11.0,12.2,16.4,18.0
5,Барселона,Парма,10.0,10.8,11.0,12.2,14.2,15.0
...,...,...,...,...,...,...,...,...
8458,Анже,Лечче,0.0,0.0,0.0,0.6,1.2,2.0
8459,Анже,Кристал Пэлас,0.0,0.0,0.0,0.6,0.6,3.0
8460,Анже,Тулуза,0.0,0.0,0.0,0.6,1.2,2.0
8461,Анже,Ноттингем Форест,0.0,0.0,0.0,0.6,1.2,2.0


In [23]:
df_att_stats = make_offsides_stats(df_5, teams_5, -7, -2, defend=False)
df_att_stats

,home,guest,Минимум,20% квантиль,Медиана,Среднее,80% квантиль,Максимум
1,Ноттингем Форест,Хетафе,3.0,5.4,8.0,8.2,10.8,14.0
2,Ноттингем Форест,Леганес,4.0,4.8,6.0,8.0,10.4,16.0
3,Ноттингем Форест,Боруссия М,5.0,5.8,6.0,8.0,9.4,15.0
4,Ноттингем Форест,Вильярреал,4.0,4.8,7.0,8.0,10.8,14.0
5,Ноттингем Форест,Фрайбург,3.0,4.6,5.0,7.8,12.4,14.0
...,...,...,...,...,...,...,...,...
8458,Милан,Астон Вилла,0.0,0.8,1.0,0.8,1.0,1.0
8459,Милан,Ливерпуль,0.0,0.0,0.0,0.8,2.0,2.0
8460,Милан,Хоффенхайм,0.0,0.0,1.0,0.8,1.2,2.0
8461,Милан,Монца,0.0,0.0,1.0,0.6,1.0,1.0


In [25]:
# Найдем результаты матчей с 18 по 21 октября
df_last_league_matches = df_5[(df_5['start_datetime'] >= '2024-10-18') & (df_5['start_datetime'] <= '2024-10-21')][['home', 'guest', 'offside_home', 'offside_guest']]
df_last_league_matches['offsides_total'] = df_last_league_matches['offside_home'] + df_last_league_matches['offside_guest']  
df_last_league_matches

,home,guest,offside_home,offside_guest,offsides_total
14189,Боруссия Д,Санкт-Паули,1.0,1.0,2.0
14190,Монако,Лилль,2.0,1.0,3.0
14191,Алавес,Вальядолид,3.0,1.0,4.0
14193,Тоттенхэм,Вест Хэм,2.0,1.0,3.0
14194,Атлетик,Эспаньол,0.0,1.0,1.0
14195,Дженоа,Болонья,3.0,3.0,6.0
14196,Комо,Парма,7.0,2.0,9.0
14197,Байер,Айнтрахт Ф,2.0,2.0,4.0
14198,Боруссия М,Хайденхайм,0.0,2.0,2.0
14199,Майнц,РБ Лейпциг,2.0,0.0,2.0


In [29]:
# Сумма
df_final_sum = pd.merge(pd.merge(df_att_stats, df_def_stats,on=['home', 'guest'], how='inner', suffixes= ('_att', '_def')),
                             df_last_league_matches, on=['home', 'guest'], how='inner')

df_final_sum['20% квантиль'] = df_final_sum['20% квантиль_att'] + df_final_sum['20% квантиль_def']
df_final_sum['Медиана'] = df_final_sum['Медиана_att'] + df_final_sum['Медиана_def']
df_final_sum['Среднее'] = df_final_sum['Среднее_att'] + df_final_sum['Среднее_def']
df_final_sum['80% квантиль'] = df_final_sum['80% квантиль_att'] + df_final_sum['80% квантиль_def']
df_final_sum = df_final_sum[['home', 'guest', '20% квантиль', 'Медиана', 'Среднее', '80% квантиль', 'offsides_total']]


# Расчет количества и доли верных прогнозов по данному методу
counter_20 = (df_final_sum['offsides_total'] > df_final_sum['20% квантиль']).value_counts()
prob_20 = round((counter_20[True] / counter_20.sum()) * 100, 2)
counter_mean = (df_final_sum['offsides_total'] > df_final_sum['Среднее']).value_counts()
prob_mean = round((counter_mean[True] / counter_mean.sum()) * 100, 2)
counter_median = (df_final_sum['offsides_total'] > df_final_sum['Медиана']).value_counts()
prob_median = round((counter_median[True] / counter_median.sum()) * 100, 2)

print(f'20% квантиль: {prob_20}%')
print(f'Медиана: {prob_median}%')
print(f'Среднее: {prob_mean}%')

df_final_sum

20% квантиль: 46.15%
Медиана: 12.82%
Среднее: 2.56%


,home,guest,20% квантиль,Медиана,Среднее,80% квантиль,offsides_total
0,Боруссия М,Хайденхайм,3.6,5.0,6.8,9.4,2.0
1,Вильярреал,Хетафе,2.6,9.0,9.4,14.6,2.0
2,Фрайбург,Аугсбург,3.2,5.0,8.2,14.8,2.0
3,Осасуна,Бетис,4.2,8.0,9.6,14.0,8.0
4,Жирона,Реал Сосьедад,4.4,6.0,7.4,9.6,2.0
5,Алавес,Вальядолид,5.4,7.0,9.6,12.4,4.0
6,Монако,Лилль,5.6,8.0,9.0,12.0,3.0
7,Лечче,Фиорентина,2.4,5.0,5.8,10.2,2.0
8,Борнмут,Арсенал,3.4,7.0,6.6,9.6,2.0
9,ПСЖ,Страсбур,5.4,9.0,9.0,12.0,3.0


**Вывод:** Средняя и медиана могут быть верхними оценками для кол-ва офсайдов матче (так как доля удачного прогноза превышения реального количества офсайдов над прогнозируемым получилась очень низким)

In [30]:
# Средняя
df_final_mean = pd.merge(pd.merge(df_att_stats,df_def_stats,on=['home', 'guest'], how='inner', suffixes= ('_att', '_def')),
                             df_last_league_matches, on=['home', 'guest'], how='inner')

df_final_mean['20% квантиль'] = np.mean((df_final_mean['20% квантиль_att'], df_final_mean['20% квантиль_def']), axis=0)
df_final_mean['Медиана'] = np.mean((df_final_mean['Медиана_att'], df_final_mean['Медиана_def']), axis=0)
df_final_mean['Среднее'] = np.mean((df_final_mean['Среднее_att'], df_final_mean['Среднее_def']), axis=0)
df_final_mean['80% квантиль'] = np.mean((df_final_mean['80% квантиль_att'], df_final_mean['80% квантиль_def']), axis=0)
df_final_mean = df_final_mean[['home', 'guest', '20% квантиль', 'Медиана', 'Среднее', '80% квантиль', 'offsides_total']]

counter_20 = (df_final_mean['offsides_total'] > df_final_mean['20% квантиль']).value_counts()
prob_20 = round((counter_20[True] / counter_20.sum()) * 100, 2)
counter_mean = (df_final_mean['offsides_total'] > df_final_mean['Среднее']).value_counts()
prob_mean = round((counter_mean[True] / counter_mean.sum()) * 100, 2)
counter_median = (df_final_mean['offsides_total'] > df_final_mean['Медиана']).value_counts()
prob_median = round((counter_median[True] / counter_median.sum()) * 100, 2)
counter_80 = (df_final_mean['offsides_total'] > df_final_mean['80% квантиль']).value_counts()
prob_80 = round((counter_80[True] / counter_80.sum()) * 100, 2)


print(f'20% квантиль: {prob_20}%')
print(f'Медиана: {prob_median}%')
print(f'Среднее: {prob_mean}%')
print(f'80% квантиль: {prob_80}%')

df_final_mean

20% квантиль: 89.74%
Медиана: 41.03%
Среднее: 33.33%
80% квантиль: 12.82%


,home,guest,20% квантиль,Медиана,Среднее,80% квантиль,offsides_total
0,Боруссия М,Хайденхайм,1.8,2.5,3.4,4.7,2.0
1,Вильярреал,Хетафе,1.3,4.5,4.7,7.3,2.0
2,Фрайбург,Аугсбург,1.6,2.5,4.1,7.4,2.0
3,Осасуна,Бетис,2.1,4.0,4.8,7.0,8.0
4,Жирона,Реал Сосьедад,2.2,3.0,3.7,4.8,2.0
5,Алавес,Вальядолид,2.7,3.5,4.8,6.2,4.0
6,Монако,Лилль,2.8,4.0,4.5,6.0,3.0
7,Лечче,Фиорентина,1.2,2.5,2.9,5.1,2.0
8,Борнмут,Арсенал,1.7,3.5,3.3,4.8,2.0
9,ПСЖ,Страсбур,2.7,4.5,4.5,6.0,3.0


**Вывод:** 20% квантиль и 80% квантиль могут быть оценками нижней и верхней оценки для кол-ва офсайдов матче

In [31]:
# Минимум
df_final_min = pd.merge(pd.merge(df_att_stats,df_def_stats,on=['home', 'guest'], how='inner', suffixes= ('_att', '_def')),
                             df_last_league_matches, on=['home', 'guest'], how='inner')

df_final_min['20% квантиль'] = np.min((df_final_min['20% квантиль_att'], df_final_min['20% квантиль_def']), axis=0)
df_final_min['Медиана'] = np.min((df_final_min['Медиана_att'], df_final_min['Медиана_def']), axis=0)
df_final_min['Среднее'] = np.min((df_final_min['Среднее_att'], df_final_min['Среднее_def']), axis=0)
df_final_min['80% квантиль'] = np.min((df_final_min['80% квантиль_att'], df_final_min['80% квантиль_def']), axis=0)
df_final_min = df_final_min[['home', 'guest', '20% квантиль', 'Медиана', 'Среднее', '80% квантиль', 'offsides_total']]

counter_20 = (df_final_min['offsides_total'] > df_final_min['20% квантиль']).value_counts()
prob_20 = round((counter_20[True] / counter_20.sum()) * 100, 2)
counter_mean = (df_final_min['offsides_total'] > df_final_min['Среднее']).value_counts()
prob_mean = round((counter_mean[True] / counter_mean.sum()) * 100, 2)
counter_median = (df_final_min['offsides_total'] > df_final_min['Медиана']).value_counts()
prob_median = round((counter_median[True] / counter_median.sum()) * 100, 2)
counter_80 = (df_final_min['offsides_total'] > df_final_min['80% квантиль']).value_counts()
prob_80 = round((counter_80[True] / counter_80.sum()) * 100, 2)


print(f'20% квантиль: {prob_20}%')
print(f'Медиана: {prob_median}%')
print(f'Среднее: {prob_mean}%')
print(f'80% квантиль: {prob_80}%')

df_final_min

20% квантиль: 94.87%
Медиана: 46.15%
Среднее: 53.85%
80% квантиль: 23.08%


,home,guest,20% квантиль,Медиана,Среднее,80% квантиль,offsides_total
0,Боруссия М,Хайденхайм,0.8,2.0,2.2,3.4,2.0
1,Вильярреал,Хетафе,0.0,2.0,2.4,3.8,2.0
2,Фрайбург,Аугсбург,1.6,2.0,3.2,4.6,2.0
3,Осасуна,Бетис,0.0,3.0,4.4,6.8,8.0
4,Жирона,Реал Сосьедад,0.8,2.0,1.8,2.4,2.0
5,Алавес,Вальядолид,1.6,2.0,4.6,6.2,4.0
6,Монако,Лилль,2.8,3.0,3.6,4.4,3.0
7,Лечче,Фиорентина,0.0,1.0,1.2,2.2,2.0
8,Борнмут,Арсенал,0.8,3.0,2.4,3.4,2.0
9,ПСЖ,Страсбур,1.8,4.0,3.8,5.4,3.0


**Вывод:** 20% квантиль и 80% квантиль могут быть оценками нижней и верхней оценки для кол-ва офсайдов матче

**ФИНАЛЬНЫЙ ВЫВОД:** проанализировав разные способы агрегации, можно сказать, что лучшей оценкой нижней границы для количества офсайдов в матче является минимум из 20% квантилей по атаке и обороне, а верхняя граница - средняя по 80% квантиль по атаке и обороне. Но это была простая проверка. Более корректно будет провести backtesting вышеназванных методов агрегации на большом количестве матчей и рассчитать вероятность как долю успешных прогнозов. 

# Backtesting

Для того чтобы, не утонуть в количестве возможных спорных моментов, при этом без значимой потери качества, ограничимся только играми внутри одной лиги, и таким образом пройдемся по ТОП-5 лигам Европы.

In [32]:
# Расчет оценок границы офсайдов в матче
def match_forecast(df, teams):
    
    forecast_def = make_offsides_stats(df, teams, -5, defend=True).iloc[0, :]
    forecast_att = make_offsides_stats(df, teams, -5, defend=False).iloc[0, :]
    
    forecast_min = min(forecast_def['20% квантиль'], forecast_att['20% квантиль'])
    forecast_max = (forecast_def['80% квантиль'] + forecast_att['80% квантиль'])/2

    return forecast_min, forecast_max

# Объединение прогнозов с реальным результатом матча
def match_analysis(df, game):
    
    teams = game[['home', 'guest']].values
    datetime = game['start_datetime']
    offsides_total = game['offside_home'] + game['offside_guest']

    forecast_min, forecast_max = match_forecast(df[df['start_datetime'] < datetime], teams)

    return teams[0], teams[1], forecast_min, forecast_max, offsides_total 

# Отбираем только одну лигу (здесь французская Лига 1) в этом сезоне без матчей 25 - 28 октября
сurr_df = df[df['region'] == "ФРАНЦИЯ"]
curr_teams = сurr_df[(сurr_df['start_datetime'] > "2024-06-15"]['home'].unique()

league_res = []

for i in range(30, сurr_df.shape[0]):
    
    game = сurr_df.iloc[i, :]
    
    try:
        league_res.append(match_analysis(сurr_df, game))
    except: 
        continue

league_df = pd.DataFrame(league_res, columns=['home', 'guest', 'forecast_min', 'forecast_max', 'offsides_total'])

# Расчёт вероятностей по команде
team_res = []

for team in curr_teams:
    
    team_df = league_df[(league_df['home'] == team) | (league_df['guest'] == team)]
    
    min_right = round((team_df['offsides_total'] > team_df['forecast_min']).mean() * 100, 2)
    max_right = round((team_df['offsides_total'] < team_df['forecast_max']).mean() * 100, 2)
    all_right = round(((team_df['offsides_total'] > team_df['forecast_min']) & ((team_df['offsides_total'] < team_df['forecast_max']))).mean() * 100, 2)

    team_res.append([team, min_right, max_right, all_right])

team_res_df = pd.DataFrame(team_res, columns=['team', 'min_right_prob', 'max_right_prob', 'all_right_prob'])

# Создаем синтетические матчи и по ним считаем считаем вероятность удачного прогноза
matches_res_df = team_res_df.merge(team_res_df, how='cross')
matches_res_df = matches_res_df[matches_res_df['team_x'] != matches_res_df['team_y']]
matches_res_df['min_right_prob'] = (matches_res_df['min_right_prob_x'] + matches_res_df['min_right_prob_y'])/2
matches_res_df['max_right_prob'] = (matches_res_df['max_right_prob_x'] + matches_res_df['max_right_prob_y'])/2
matches_res_df['all_right_prob'] = (matches_res_df['all_right_prob_x'] + matches_res_df['all_right_prob_y'])/2
matches_res_df = matches_res_df[['team_x', 'team_y', 'min_right_prob', 'max_right_prob', 'all_right_prob']]

# Добавим прогноз на следующий матч (то есть на матчи 25-28 октября)
forecast_min = []
forecast_max = []

for i in matches_res_df.values:
    forecasts = match_forecast(сurr_df, [i[0], i[1]])
    forecast_min.append(forecasts[0])
    forecast_max.append(forecasts[1])

matches_res_df['forecast_min'] = forecast_min
matches_res_df['forecast_max'] = forecast_max

Далее необходимо запустить данную функцию по каждой лиге и отобрать матчи, где наблюдается тенденция делать много или мало офсайдов. А далее я вручную сравниваю эту вероятность с букмекерской в попытке найти отличия в моих и букмекерских оценках. Собственно, ниже вы видите алгоритм именно такого поиска матчей по играм с 25-28 октября. 

In [7]:
matches_res_df_eng = matches_res_df 
matches_res_df_eng[(matches_res_df_eng['team_x'] == 'Эвертон') & (matches_res_df_eng['team_y'] == 'Фулхэм')]
matches_res_df_eng[(matches_res_df_eng['team_x'] == 'Манчестер Сити') & (matches_res_df_eng['team_y'] == 'Саутгемптон')]

In [14]:
matches_res_df_esp = matches_res_df
matches_res_df_esp[(matches_res_df_esp['team_x'] == 'Реал Сосьедад') & (matches_res_df_esp['team_y'] == 'Осасуна')]

In [21]:
matches_res_df_ger = matches_res_df

In [25]:
matches_res_df_itl = matches_res_df
matches_res_df_itl[(matches_res_df_itl['team_x'] == 'Интер') & (matches_res_df_itl['team_y'] == 'Ювентус')]

In [33]:
matches_res_df_fra = matches_res_df
matches_res_df_fra[(matches_res_df_fra['team_x'] == 'Ланс') & (matches_res_df_fra['team_y'] == 'Лилль')]

Результат

In [39]:
pd.concat([matches_res_df_eng[(matches_res_df_eng['team_x'] == 'Эвертон') & (matches_res_df_eng['team_y'] == 'Фулхэм')],
           matches_res_df_eng[(matches_res_df_eng['team_x'] == 'Манчестер Сити') & (matches_res_df_eng['team_y'] == 'Саутгемптон')],
           matches_res_df_esp[(matches_res_df_esp['team_x'] == 'Реал Сосьедад') & (matches_res_df_esp['team_y'] == 'Осасуна')],
           matches_res_df_itl[(matches_res_df_itl['team_x'] == 'Интер') & (matches_res_df_itl['team_y'] == 'Ювентус')],
           matches_res_df_fra[(matches_res_df_fra['team_x'] == 'Ланс') & (matches_res_df_fra['team_y'] == 'Лилль')]], ignore_index=True)

,team_x,team_y,min_right_prob,max_right_prob,all_right_prob,forecast_min,forecast_max
0,Эвертон,Фулхэм,87.365,78.725,66.095,2.4,7.4
1,Манчестер Сити,Саутгемптон,86.605,78.155,64.760,0.0,2.5
2,Реал Сосьедад,Осасуна,90.985,74.230,65.215,2.6,7.9
3,Интер,Ювентус,84.605,81.980,66.585,0.0,2.8
4,Ланс,Лилль,86.255,73.850,60.105,2.8,5.3


**Вывод:** Данные матчи уже сыграны, и все мои прогнозы по офсайдам оказались верными!